In [41]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from newspaper import Article
from tqdm import tqdm

In [42]:
muc = 'thoi-su'
max_page = 5542

In [43]:
def get_title():
    title_1 = soup2.find('div', class_ ='verticalPost__main')
    if title_1 != None:
        heading_2 = title_1.find('h2')
        title1 = heading_2.find('a').get('title')
        titles.append(title1)

    title_2 = soup2.find_all('h3')
    for tit in title_2:
        title = tit.find('a').get('title')
        titles.append(title)

In [44]:
def get_category():
    vert_cat = soup2.find_all('div', class_='verticalPost__main-cate')
    if vert_cat != None:
        for cat in vert_cat:
            category1 = cat.find('a').get('title')
            cate.append(category1)
            
    hor_cat = soup2.find_all('div', class_='horizontalPost__main-cate show')
    for cat in hor_cat:
        category2 = cat.find('a').get('title')
        cate.append(category2)

In [45]:
def get_url():
    for links in vertical_post:
        anchor = links.find('a')
        link = 'https://vietnamnet.vn/'+ anchor.attrs['href']
        # link =  anchor.attrs['href']
        urls.append(link)
    for links in horizontal_post:
        anchor = links.find('a')
        link = 'https://vietnamnet.vn/' + anchor.attrs['href']
        # link = anchor.attrs['href']
        urls.append(link)

In [46]:
def get_url_2():
    for links in vertical_post:
        anchor = links.find('a')
        # link = 'https://vietnamnet.vn/'+ anchor.attrs['href']
        link =  anchor.attrs['href']
        urls.append(link)
    for links in horizontal_post:
        anchor = links.find('a')
        # link = 'https://vietnamnet.vn/' + anchor.attrs['href']
        link = anchor.attrs['href']
        urls.append(link)

In [47]:
def get_article_type_1():
        # article = soup.find('div', class_ = 'container__left not-pl')
        body = article.find(class_ = 'maincontent main-content').text
        time = soup.find('div', class_ = 'bread-crumb-detail__time').text.strip()
        date.append(time)
        text.append(body)
    

In [48]:
def get_article_type_2():
    time = soup.find('div', class_ = 'bread-crumb-detail__time').text.strip()
    body = soup.find_all('p')
    for txt in body: 
        text.append(txt.get_text())


In [49]:
news_df = pd.DataFrame(columns= ['date', 'title', 'category', 'text', 'url'])

In [51]:
print('Đang cào mục', muc)
for page in tqdm(range(1, 4), ncols=100):
    try:
        titles = []
        cate = []
        urls = []
        date = []
        text = []
        request = requests.get(f'https://vietnamnet.vn/{muc}-page{page}')
        request = request.content
        soup2 = BeautifulSoup(request,'html.parser')
        vertical_post = soup2.find_all('div', class_ = 'verticalPost__avt')
        horizontal_post = soup2.find_all('div', class_ = 'horizontalPost__avt avt-240')
        # Get article titles
        get_title()
        # Get article categories 
        get_category()
        # Get article links
        get_url()
        # Scrape an article
        for url in urls:
            try:
                req = requests.get(url)
                req = req.content
                soup = BeautifulSoup(req, 'html.parser')
                article = soup.find('div', class_ = 'container__left not-pl')
                if article != None:
                    try:
                        get_article_type_1()   
                    except:
                        continue 
                else:
                    try:
                        get_article_type_2()
                    except Exception as error:
                        print(url, error)
            except Exception as error:
                        print(url, error)
        news_list_1 = pd.DataFrame(list(zip(date, titles, cate, text, urls)), columns= ['date', 'title', 'category', 'text', 'url'])
        news_df = pd.concat([news_df, news_list_1])
    except Exception as error:
         print(error)

Đang cào mục thoi-su


  0%|                                                                         | 0/3 [00:00<?, ?it/s]

In [ ]:
print('Đang cào mục', muc)
for page in tqdm(range(4, max_page + 1), ncols=100):
    try:
        titles = []
        cate = []
        urls = []
        date = []
        text = []
        request = requests.get(f'https://vietnamnet.vn/{muc}-page{page}')
        request = request.content
        soup2 = BeautifulSoup(request,'html.parser')
        vertical_post = soup2.find_all('div', class_ = 'verticalPost__avt')
        horizontal_post = soup2.find_all('div', class_ = 'horizontalPost__avt avt-240')
        # Get article titles
        get_title()
        # Get article categories 
        get_category()
        # Get article links
        get_url_2()
        # Scrape an article
        for url in urls:
            try:
                req = requests.get(url)
                req = req.content
                soup = BeautifulSoup(req, 'html.parser')
                article = soup.find('div', class_ = 'container__left not-pl')
                if article != None:
                    try:
                        get_article_type_1()   
                    except:
                        continue 
                else:
                    try:
                        get_article_type_2()
                    except Exception as error:
                        print(url, error)
            except Exception as error:
                        print(url, error)
        news_list_2 = pd.DataFrame(list(zip(date, titles, cate, text, urls)), columns= ['date', 'title', 'category', 'text', 'url'])
        news_df = pd.concat([news_df, news_list_2])
    except Exception as error:
         print(error)

In [ ]:
# news_list = list(zip(date, titles, cate, text, urls))

In [ ]:
# print(news_list)

In [ ]:
news_df.to_csv(f'./vietnamnet-{muc}.csv')